In [32]:
from mlflow_extra.loggers.code import check_logged_code
from mlflow_extra.loggers.code import append_code
from mlflow_extra.tests.testing_functions import do_something
from mlflow_extra.experiments.utils import get_or_create_experiment
import tempfile
import os
import inspect

In [5]:
experiment = get_or_create_experiment("example_experiment")

In [6]:
import mlflow

In [21]:

run = mlflow.get_run("3bed9459ac104a1baaf011f541afdc28")

In [28]:
module_name = "mlflow_extra/tests/testing_functions.py"
mlflow.artifacts.download_artifacts(run_id = run.info.run_id,artifact_path=module_name, dst_path=".")

'c:\\Users\\manue\\projects\\mlflow_extra\\notebooks\\mlflow_extra/tests/testing_functions.py'

In [41]:
def add(a,b):
    return a+b

In [42]:
import os
def append_code(run, func):
    module_name = func.__module__
    with tempfile.TemporaryDirectory() as tmpdirname:
        mlflow.artifacts.download_artifacts(run_id = run.info.run_id, artifact_path= "mlflow_extra/tests/testing_functions.py", dst_path=tmpdirname)
        # print(os.listdir(tmpdirname))
        with open(tmpdirname + "/" + "mlflow_extra/tests/testing_functions.py", "r") as f:
            code = f.read()
            func_code = inspect.getsource(func)
            print(func_code)
            full_code = "\n\n".join([code, func_code])
    return full_code
full_code = append_code(run, add)        


def add(a,b):
    return a+b



In [50]:
from pathlib import Path

module_path = Path("mlflow_extra/tests/testing_functions.py")

files = mlflow.artifacts.list_artifacts(run_id = run.info.run_id, artifact_path= module_path.parent)

In [54]:
[file.path for file in files]

['mlflow_extra/tests/testing_classes.py',
 'mlflow_extra/tests/testing_functions.py']

In [60]:
module_path.as_posix()

'mlflow_extra/tests/testing_functions.py'